<a href="https://colab.research.google.com/github/jaredlan1/getting_started_with_ML/blob/main/PINN_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



In [11]:
class MLP(nn.Module):
  def __init__(self, hlayers , width):
    super(MLP, self).__init__()

    layers = []
    layers.append(nn.Linear(1, width))
    layers.append(nn.Tanh())

    for _ in range(hlayers-1):
      layers.append(nn.Linear(width,width))
      layers.append(nn.Tanh())

    layers.append(nn.Linear(width,1))

    self.network = nn.Sequential(*layers)

  def forward(self, t): # t is size 1,1
       return self.network(t)

In [9]:
def residual(model, t, params): # t size:(ncol, 1)

  #unpack parameters
  m, mu, k = params
  #evaluate model
  y = model(t) # size (ncol,1)

  #compute derivatives
  dydt = torch.autograd.grad(y, t,
                             grad_outpust = torch.ones_like(y),
                             create_graph=True)[0] # (ncol,1)

  dy2ydt2 = torch.autograd.grad(dydt, y,
                            grad_outpust = torch.ones_like(y),
                            create_graph=True)[0] #(ncol,1)

  #compute residual
  return m*dy2ydt2 + mu*dydt + k*y


In [12]:
m=1; mu=4; k=400

params = (m, mu, k)

hlayers = 4; width = 32
model = MLP(hlayers, width)

In [ ]:
def boundary(model, tbc): #tbc: (nbc, 1) time tensor boundary condition

  #evaluate model
  ybc = model(tbc)

  # compute derivatives

  dydt = torch.autograd.grad(ybc, tbc,
                             grad_outpust = torch.ones_like(ybc),
                             create_graph=True)[0] # (ncol,1)
  # compute residuals

  return ybc - 1, dydt

In [ ]:
def datapoints():

    tdata = torch.zeros(1,1)
    tcol = torch.linspace(0, 1, 50).reshape(50, 1)